In [346]:

import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

# Read the source data.
in_filename = 'https://s3.eu-west-1.amazonaws.com/data.defra.gov.uk/Waste/Single_use_carrier_bag_England_data_2016_17_to_2022_23.csv'
df = pd.read_csv(in_filename, encoding='cp1252', thousands=',')

'''
Cleaning time
'''

# Company names can be wrong, case in point, Aldi|ALDI.abs
# df[df['Company name'].str.contains("Aldi|ALDI")]
df['Company name'] = df['Company name'].str.capitalize()

# Trim all the leading/trailing whitespace from the column names.
df.rename(columns=lambda x: x.strip(), inplace=True)

# Shorten the really long column name
bag_column_rename = {
    "Number of single use plastic bags issued": "Single use plastic bags",
    "Number of paper bags issued": "Paper bags",
    "Number of bags for life issued": "Bags for life",
    "Number of fabric bags issued": "Fabric bags",
    "Number of other bags issued": "Other bags"
}
df.rename(columns=bag_column_rename, inplace=True)

# select specific columns, the bagsColumn has whitespace :(
plastic_bags_col = bag_column_rename["Number of single use plastic bags issued"]
# It doesn't really matter than null exists but we want a value for the charts.
df.fillna({plastic_bags_col: 0}, inplace = True)

# Replace their empty symbol (:) with 0, remove commas and cast  to number
for x in bag_column_rename.values():
    df[x] = df[x].replace(to_replace=',', value='', regex=True).replace(to_replace=':', value=0)
    df[x] = df[x].astype(int)


# Rename/cast the year values to just the year.
df['Year'] = df['Year'].str.slice(0, 4).astype(int)

# Create a smaller working set of needed columns
workingset = df[["Year", "Company name", plastic_bags_col, "Paper bags", "Bags for life", "Fabric bags", "Other bags"]]

workingset.head()


,Year,Company name,Single use plastic bags,Paper bags,Bags for life,Fabric bags,Other bags
0,2016,2 sisters (red meat) ltd,20638,0,0,0,0
1,2016,A f blakemore & son ltd,6439197,0,0,0,0
2,2016,Age uk trading cic,818423,0,0,0,0
3,2016,Aldi stores ltd,67203919,0,3750662,0,0
4,2016,"Animal, a division of h young (operations) lim...",30546,0,0,0,0


In [347]:
'''
At first, I tried getting the top 5 per year this meant we had more than 5 unique companies, and if companies didn't make the
top 5 consistenly, their failing years would be NaN.

This means, it made more sense to have a single table with the top 5 sum overall.
'''

# Show an example of the problem with using the top 3 per year
topten = workingset[["Year", "Company name", plastic_bags_col]]
#topten = topten[["Year", "Company name", plastic_bags_col]]
topten = topten.sort_values(by=["Year", plastic_bags_col], ascending=[True, False])
topten = topten.groupby('Year', as_index=False).nth[:3]

print("Asda only have three entries when selecting the top 3 per year")
print(topten[topten['Company name'].str.contains("Asda")])
print("-----------\r\n")
print("Asda have at least 6 entries when not filtered")
all_topten = workingset[["Year", "Company name", plastic_bags_col]]
print(all_topten[all_topten['Company name'].str.contains("Asda")].head(6))
print("-----------\r\n")




Asda only have three entries when selecting the top 3 per year
     Year         Company name  Single use plastic bags
7    2016  Asda stores limited                165766608
268  2017  Asda stores limited                192973018
-----------

Asda have at least 6 entries when not filtered
      Year         Company name  Single use plastic bags
7     2016  Asda stores limited                165766608
268   2017  Asda stores limited                192973018
516   2018  Asda stores limited                106896430
735   2019  Asda stores limited                 20114385
927   2020  Asda stores limited                 64719465
1093  2021  Asda stores limited                  5661199
-----------



In [348]:
# Get the top total bag users, regardless of materials (this gets the most prolific companies)
bag_columns = [
  "Single use plastic bags",
  "Paper bags",
  "Bags for life",
  "Fabric bags",
  "Other bags"
]
total = workingset.groupby(['Company name'])[bag_columns].sum()
total['Total'] = total[bag_columns].sum(axis=1)
total = total.sort_values(by='Total', ascending=False).reset_index()

total = total.head(5)
total.to_json('./data/top5_totals.json', orient='records')

total.head(10)


,Company name,Single use plastic bags,Paper bags,Bags for life,Fabric bags,Other bags,Total
0,Tesco stores ltd,1147121371,0,0,0,0,1147121371
1,Ocado retail limited,1095530317,0,0,0,0,1095530317
2,The co-operative group,734693851,0,130405878,5167676,0,870267405
3,Wm morrison supermarkets plc,706992017,0,0,0,0,706992017
4,Asda stores limited,556252661,0,0,0,0,556252661


In [349]:
# Total top 5 grouped by Company, Summed and Sorted
top_df = workingset.groupby('Company name', as_index=False)[plastic_bags_col].sum()
top_df = top_df.sort_values(by=plastic_bags_col, ascending=False, inplace=False)
top_df = top_df.head(5)
# Filter the workingest data to just the rows we need
toptier = workingset[workingset['Company name'].isin(top_df['Company name'])].reset_index()

toptier.to_json('./data/top5_summary.json', orient='records')

toptier.head(10)

,index,Year,Company name,Single use plastic bags,Paper bags,Bags for life,Fabric bags,Other bags
0,7,2016,Asda stores limited,165766608,0,0,0,0
1,159,2016,Ocado retail limited,128388732,0,0,0,0
2,211,2016,Tesco stores ltd,637181382,0,0,0,0
3,215,2016,The co-operative group,125929333,0,19913286,0,0
4,256,2016,Wm morrison supermarkets plc,191508650,0,0,0,0
5,268,2017,Asda stores limited,192973018,0,0,0,0
6,406,2017,Ocado retail limited,140793060,0,0,0,0
7,459,2017,Tesco stores ltd,332741451,0,0,0,0
8,463,2017,The co-operative group,120629924,0,19887722,0,0
9,502,2017,Wm morrison supermarkets plc,183271200,0,0,0,0


In [350]:
# Pivot to column names, better for JSON.
pivot = toptier.pivot(index='Year', columns="Company name", values=plastic_bags_col).reset_index()
pivot = pivot.fillna(0)
pivot.to_json('./data/top5.json', orient='records')
pivot.head(10)


Company name,Year,Asda stores limited,Ocado retail limited,Tesco stores ltd,The co-operative group,Wm morrison supermarkets plc
0,2016,165766608.0,128388732.0,637181382.0,125929333.0,191508650.0
1,2017,192973018.0,140793060.0,332741451.0,120629924.0,183271200.0
2,2018,106896430.0,156311434.0,121749784.0,112602662.0,65071608.0
3,2019,20114385.0,163280210.0,42117456.0,106578322.0,55449507.0
4,2020,64719465.0,112600000.0,11889176.0,95462928.0,94946130.0
5,2021,5661199.0,202142190.0,1442122.0,104375036.0,59565625.0
6,2022,121556.0,192014691.0,0.0,69115646.0,57179297.0


In [351]:
# Total values per year
annual_totals = pd.DataFrame()
for x in bag_column_rename.values():
    annual_totals[x] = workingset.groupby('Year')[x].sum()

annual_totals = annual_totals.reset_index()
annual_totals.to_json('./data/annual_totals.json', orient='records')
annual_totals


,Year,Single use plastic bags,Paper bags,Bags for life,Fabric bags,Other bags
0,2016,2116049476,20652893,70020737,0,0
1,2017,1754130932,1987407,39870379,0,0
2,2018,1126771078,4327110,114213874,0,0
3,2019,627334384,2773709,132454982,603765,0
4,2020,488545004,837371,33230904,492120,1213383
5,2021,496236498,1817191,59340929,1658958,876772
6,2022,405975904,733754,3166811,2921894,1342036
